In [3]:
import numpy as np
import scipy.integrate as spint
import scipy.interpolate as spinter
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

In [23]:
class StreamLines:
    """
    startPos is a (N,D) array, where N is the number of streamlines and D is the number of dimensions (e.g.: 3)
    fieldFunc is the function f(t,x)->dx/dt where x is of dimension D
    """
    def __init__(self, startPos, fieldFunc, t_max, tN=200, tol=1e-5, showRatio=0.2):
        self.startPos = startPos
        self.fieldFunc = fieldFunc
        self.t_max = t_max
        self.rtol = tol
        self.atol = tol*1e-2
        self.tN = tN
        self.showN = int(tN*showRatio)
    
    def getLinesAtTime(self,t,fadeIn=True,fadeOut=True):
        fI = 0
        fO = 0
        if fadeIn:
            fI = self.showN
        if fadeOut:
            fO = self.showN

        startT = int((self.tN-1+fI-self.showN+fO)*t/self.t_max)-fI
        endT = startT+self.showN
        return self.lines[:,np.clip(startT,0,self.tN-1):np.clip(endT,0,self.tN-1),:], self.cvals[:,np.clip(startT,0,self.tN-2):np.clip(endT-1,0,self.tN-2)]

    def compute(self):
        t_evals = np.linspace(0,self.t_max,self.tN)
        self.lines = np.empty((self.startPos.shape[0],self.tN,self.startPos.shape[1]))
        self.cvals = np.empty((self.startPos.shape[0],self.tN-1))

        for i in range(len(self.startPos)):
            rkdp = spint.RK45(self.fieldFunc, 0.0, self.startPos[i], self.t_max, first_step=self.rtol*1e-2,rtol=self.rtol,atol=self.atol)
            ts = [0.0]
            zs = [rkdp.y]
            while rkdp.t < self.t_max:
                rkdp.step()
                ts.append(rkdp.t)
                zs.append(rkdp.y)
                if rkdp.status != "running":
                    break
            zs = np.array(zs).T
            intf = spinter.interp1d(ts, zs, fill_value=(zs[:,0],zs[:,-1]), bounds_error=False)
            self.lines[i,:,:] = intf(t_evals).T
            self.cvals[i,:] = np.sqrt(np.sum((self.lines[i,1:,:]-self.lines[i,:-1,:])**2, axis=1))

In [90]:
def efield(_t,rv):
    r = np.linalg.norm(rv)
    if r < 1e-10:
        return 0*rv
    return -rv/r**3

def bfield(_t,rv):
    r = np.linalg.norm(rv)
    if r < 1e-10:
        return 0*rv
    S = np.array([0,0,1])
    return (S-3*np.dot(S,rv)/r**2 * rv)/r**3

r = 2.6247
sps = np.array([[r*np.cos(phi_i*np.pi*2),r*np.sin(phi_i*np.pi*2),2.2085] for phi_i in np.arange(0,1,0.25)])
sls = StreamLines(sps, bfield, 24, tN=10000, tol=1e-6, showRatio=0.2)
sls.compute()
print(sls.lines.shape)
print(sls.lines[:,-1,:])

(4, 10000, 3)
[[ 9.22190243e-06  0.00000000e+00  7.92725015e-04]
 [ 5.64678665e-22  9.22190243e-06  7.92725015e-04]
 [-9.22190243e-06  1.12935733e-21  7.92725015e-04]
 [-1.69403599e-21 -9.22190243e-06  7.92725015e-04]]


In [91]:
%matplotlib qt

min_l = -3
max_l = 3

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

fig.subplots_adjust(bottom=0.25)

ax.set_aspect('equal')
ax.set(xlim=[min_l,max_l], ylim=[min_l,max_l], zlim3d=[min_l,max_l], xlabel=r'x', ylabel=r'y')
ax.view_init(elev=10., azim=(-100))

# pnt_objs = []
# for line in sls.lines:
#     pnts = line.T
#     pnt_obj = ax.scatter(pnts[0],pnts[1],pnts[2], marker=".",color="k",zorder=10)
#     pnt_objs.append(pnt_obj)

t_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03])
t_slider = Slider(t_slider_ax, 't', 0, sls.t_max, valstep=sls.t_max/100, valinit=0)

angle_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
angle_slider = Slider(angle_slider_ax, 'angle', 0, 360, valinit=0)

def t_on_changed(t_val):
    fig.suptitle(rf'$t={t_val:0.2f}$')
    ax.cla()
    ax.set_aspect('equal')
    ax.set(xlim=[min_l,max_l], ylim=[min_l,max_l], zlim3d=[min_l,max_l], xlabel=r'x', ylabel=r'y')
    lines, cvals = sls.getLinesAtTime(t_val)
    for line in lines:
        ax.plot(line[:,0],line[:,1],line[:,2])
    fig.canvas.draw()
def angle_on_changed(val):
    angle = angle_slider.val
    ax.view_init(elev=10., azim=(-angle-100))
    fig.canvas.draw_idle()
t_slider.on_changed(t_on_changed)
angle_slider.on_changed(angle_on_changed)

plt.show()

In [31]:
print(1.1%1.0)

0.10000000000000009
